In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch

from mvn.utils.minimon import MiniMon
from mvn.utils.misc import flush_cache

from mvn.ipynb import get_args, get_config, build_labels
from train import setup_dataloaders, setup_experiment, build_env, do_train


flush_cache()

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [3]:
is_distributed = False
master = True
device = torch.device(0)
print('using dev {}'.format(device))
    
args = get_args()

# just to get a feeling of the dataset
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10000)
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10, allowed_subjects=['S9', 'S11'])

using dev cuda:0
# available GPUs: 1


In [15]:
config = get_config(args)
do_train(None, None, config, device, is_distributed, master)

conv1 9408
bn1 128
relu 0
maxpool 0
layer1 147968
layer2 525568
layer3 2099712
layer4 8393728
alg_confidences 3808529
deconv_layers 295104
final_layer 561
15280706 15280706
  training dataset length: 3
  validation dataset length: 3
Experiment name: human36m_alg_AlgebraicTriangulationNet@29.04.2021-17:45:09
epoch    0 has started!
  training batch iter 0 loss ~ 3681.866
  training MPJPE relative to pelvis: 1056.270 mm
  eval MPJPE relative to pelvis: 496.785 mm
epoch    1 has started!
  training batch iter 0 loss ~ 3448.446
  training MPJPE relative to pelvis: 993.475 mm
  eval MPJPE relative to pelvis: 497.511 mm
    evaluate results x          4 ~        0.0 [min:        0.0,    max:        0.0,    last:        0.0]
       alg: backbone x          4 ~        0.0 [min:        0.0,    max:        0.0,    last:        0.0]
   alg: tri in world x          4 ~        0.0 [min:        0.0,    max:        0.0,    last:        0.0]
           calc loss x          2 ~        0.0 [min:        